In [0]:
import re
import os
import random
import time
import binascii
import sys

fstart = time.time()
numHashes = 10

numDocs = 10000
numElems = int(numDocs * (numDocs - 1) / 2)


jacard = [0 for j in range(numElems)]
minHashSim = [0 for i in range(numElems)]

dataFile = 'docs_'+str(numDocs) + '.train'
plagFile = 'docs_'+str(numDocs) +'.plag'


#returns index of triangular matrix
def get_index(i,j):
    #k = i*numDocs + j
    k = int(i * (numDocs - (i + 1) / 2.0) + j - i) - 1
    
    return k

doc_file = open(dataFile,'r')
doc = doc_file.read()

doc = list(doc.split('\n'))


doc = doc[:numDocs]


plagiarisms = {}

f = open(plagFile,'r')
for line in f:
    if line[-1] == '\n':
        line = line[0:-1]
    
    l = line.split(' ')
    plagiarisms[l[0]] = l[1]
    plagiarisms[l[1]] = l[0]
    

In [0]:
def generate_shingles(s, n=3):
    tokens = [token for token in s.split(" ")]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return ["".join(ngram) for ngram in ngrams]

In [0]:

print('Creating Shingles')

c_shingleID = 0
documentShingleSets = {}

doc_names = []
n_shingles = 0

shingle_Set = set()

#chapter 6 in book frequent item (traingular matrix implementation)

for i in range(len(doc)):

    row = doc[i].split(' ',1)
    
    shingles = generate_shingles(row[1])
    shingle_hash = set()
#     words = row[1].split(' ')
    
    for shingle in shingles:
        h = binascii.crc32(shingle.encode()) & 0xffffffff

        shingle_hash.add(h)
        shingle_Set.add(h)
    
    doc_names.append(row[0])
    
    n_shingles += len(shingles)
    
    documentShingleSets[row[0]] = shingle_hash
    
    
    
    
print(n_shingles,len(shingle_Set))
print('Average shingles : ',n_shingles/numDocs)
# print(doc_names)


Creating Shingles
2519893 1540681
Average shingles :  251.9893


In [0]:
start = time.time()

for i in range(numDocs):
#     l = []
    for j in range(i+1,numDocs):
        s1 = documentShingleSets[doc_names[i]]
        s2 = documentShingleSets[doc_names[j]]
#         l.append(len(s1&s2)/(len(s1|s2)))

        jacard[(int)(get_index(i,j))] = (len(s1&s2)/len(s1|s2))
    
finish = time.time()
print('Time taken to create Jacard : ',finish-start)
#for using the triangular matrix
# k = (i-1)(n-i/2) + j- i


        
        
        

Time taken to create Jacard :  9.429308891296387


In [0]:
#Min Hashing
maxShingleID = pow(2,32)-1

mod = 4294967311


#(ax+b)%c

#generating the random coefficients



def generate_coef(k):
    
    i = 0
    l = []
    while i<k:
        x = random.randint(0,maxShingleID)
        while x in l:
            x = random.randint(0,maxShingleID)
            
        l.append(x)
        i += 1
        
    return l
            
        
        



In [0]:

A = generate_coef(numHashes)
B = generate_coef(numHashes)

signatures = []
start = time.time()
for doc in doc_names:
    shingles = documentShingleSets[doc]
    
    sign = []
    
    for i in range(numHashes):
        m = sys.maxsize
        
        for shingle in shingles:
            h = ((A[i]*shingle)%mod + B[i])%mod
            
            if m>h:
                m = h
        sign.append(m)
    signatures.append(sign)
    
finish = time.time()
print('Time taken to create signatures : ',finish-start)


    
    
    
    
            
        



Time taken to create signatures :  10.49237322807312


In [0]:

start = time.time()
for i in range(numDocs):
    sign1 = signatures[i]
    for j in range(i+1,numDocs):
        sign2 = signatures[j]
        c = 0
        for k in range(len(sign2)):
            if sign1[k] == sign2[k]:
                c += 1
                
        
        
        minHashSim[int(get_index(i,j))] = c / numHashes

finish = time.time()

print('Time taken to generate MinHash : ',finish-fstart)
# print(minHashSim)        

Time taken to generate MinHash :  128.63265800476074


In [0]:
true_pos = 0
false_pos = 0
threshold = 0.5


print('Similar documents : \n DOC1 DOC2 MINHASH JACARD')
for i in range(numDocs):
    for j in range(i+1,numDocs):
        if minHashSim[int(get_index(i,j))] > threshold:
            print(doc_names[i],doc_names[j],minHashSim[int(get_index(i,j))],jacard[int(get_index(i,j))])
            
            if doc_names[i] in plagiarisms and plagiarisms[doc_names[i]] == doc_names[j]:
                true_pos += 1
            else:
                false_pos += 1
                
                
finish = time.time()               
print('True positive : ',true_pos,' / ',len(plagiarisms)//2)
print('False positive : ',false_pos)

print('Time taken to run the complete code : ',finish-fstart)
            
            

Similar documents : 
 DOC1 DOC2 MINHASH JACARD
t6520 t6906 1.0 0
t980 t2023 1.0 0
t9248 t4211 1.0 0
t4969 t2390 1.0 0
t8090 t1898 1.0 0
t379 t3446 1.0 0
t8805 t8306 1.0 0
t6092 t3783 1.0 0
t1513 t764 1.0 0
t5442 t906 1.0 0
t4530 t7907 0.9 0
t8826 t9285 0.9 0
t3136 t8469 1.0 0
t1726 t9170 1.0 0
t5239 t2001 1.0 0
t7958 t1621 1.0 0
t104 t4172 1.0 0
t2475 t1142 1.0 0
t3072 t7923 1.0 0
t7563 t3466 1.0 0
t673 t2432 1.0 0
t8535 t448 1.0 0
t6613 t9385 1.0 0
t6261 t5262 1.0 0
t1768 t5248 0.9 0
t3268 t7998 1.0 0
t6205 t4467 1.0 0
t1782 t7716 1.0 0
t7527 t8101 0.8 0
t288 t6999 1.0 0
t4591 t1206 1.0 0
t617 t3684 1.0 0
t969 t6244 0.8 0
t9445 t6370 1.0 0
t1088 t5015 1.0 0
t4015 t2356 1.0 0
t5999 t1403 1.0 0
t1436 t492 1.0 0
t797 t3088 1.0 0
t9363 t1012 0.9 0
t8496 t4615 1.0 0
t3600 t644 1.0 0
t7412 t7623 1.0 0
t2535 t8642 1.0 0
t4638 t1297 1.0 0
t6571 t2100 1.0 0
t4022 t3358 1.0 0
t462 t7069 1.0 0
t9596 t787 1.0 0
t269 t8413 1.0 0
t4099 t3725 1.0 0
t5551 t7693 1.0 0
t4792 t7973 1.0 0
t2839 t9303 1.0